## Introduction
Greetings from the Kaggle bot! This is an automatically-generated kernel with starter code demonstrating how to read in the data and begin exploring. If you're inspired to dig deeper, click the blue "Fork Notebook" button at the top of this kernel to begin editing.

## Exploratory Analysis
To begin this exploratory analysis, first import libraries and define functions for plotting the data using `matplotlib`. Depending on the data, not all plots will be made. (Hey, I'm just a simple kerneling bot, not a Kaggle Competitions Grandmaster!)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score,adjusted_mutual_info_score
from umap import UMAP
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


There are 2 csv files in the current version of the dataset:


In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
data_path = '/kaggle/input/data.csv'
labels_path = '/kaggle/input/labels.csv'

The next hidden code cells define functions for plotting data. Click on the "Code" button in the published kernel to reveal the hidden code.

In [ ]:
data = pd.read_csv (data_path, index_col=0)
nRow, nCol = data.shape
print (f'There are {nRow} rows and {nCol} columns in the data.')

In [ ]:
data.head()

In [ ]:
labels = pd.read_csv(labels_path)
nRow, nCol = labels.shape
print (f'There are {nRow} rows and {nCol} columns in the labels data.')

In [ ]:
labels.head()

In [ ]:
# identify unoque classes/labels in the labels data
labels['Class'].unique()

In [ ]:
#Create a 2D numpy array of values in data.
X=data.values
X[0:5]

In [ ]:
#Standardize the features before performing dimensionality reduction, (mean=0,standard deviation =1)
X_std = StandardScaler().fit_transform(X)

In [ ]:
#Visualize data using Principal Component Analysis.
print ('pca analysis')
pca = PCA(n_components=2).fit_transform(X_std)
pca_df = pd.DataFrame(data = pca, columns=['PC1', 'PC2']).join(labels)
palette = sns.color_palette('muted', n_colors=5)
sns.set_style("white")
sns.scatterplot(x='PC1', y='PC2', data=pca_df, hue='Class', palette=palette, linewidth=0.2, alpha=0.1, s=30).set_title('PCA')

In [ ]:
# fit pca on data (how much variance explained by pca)
pca_std=PCA().fit(X_std)
percent_variance = pca_std.explained_variance_ratio_*100

#Plotting Cumulative Summation of the Explained Variance
plt.figure()
plt.plot(np.cumsum(pca_std.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Cancer Dataset - Cumulative Explained Variance')
plt.show()

In [ ]:
#Visualize data using t-SNE.
print ('tsne visualisation')
model=TSNE(learning_rate=10, n_components=2, random_state=123, perplexity=30)
tsne=model.fit_transform(X_std)
tsne_df=pd.DataFrame(data=tsne, columns=['t-SNE1', 't-SNE2']).join(labels)
palette=sns.color_palette('muted', n_colors=5)
sns.set_style('white')
sns.scatterplot(x='t-SNE1', y='t-SNE2', data=tsne_df, s=30, alpha=0.2, palette=palette, linewidth=0.2, hue='Class').set_title('t-SNE')


In [ ]:
#Visualize data using t-SNE after PCA.
print("tsne on pca")
X_reduced = PCA(n_components=100).fit_transform(X_std)
model=TSNE(learning_rate=10, n_components=2, random_state=123, perplexity=30)
tsne_pca=model.fit_transform(X_reduced)
tsne_pca_df=pd.DataFrame(tsne_pca, columns=['t-SNE1', 't-SNE2']).join(labels)
palette=sns.color_palette#Visualize data using UMAP.
print("Uniform Manifold Approximation and Projection (UMAP)")
model = UMAP(n_neighbors = 40, min_dist = 0.4, n_components = 2)
umap = model.fit_transform(X_std)
umap_df = pd.DataFrame(data=umap, columns=['UMAP1','UMAP2']).join(labels)
palette = sns.color_palette("muted", n_colors=5)
sns.set_style("white")
sns.scatterplot(x='UMAP1',y='UMAP2',hue='Class',data=umap_df, palette=palette, linewidth=0.2, s=30, alpha=1).set_title('UMAP')('muted', n_colors=5)
sns.set_style('white')
sns.scatterplot(x='t-SNE1', y='t-SNE2', hue='Class', data=tsne_pca_df, s=30, alpha=0.1, linewidth=0.2).set_title('tsne on pca')


In [0]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()


In [0]:
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()


In [0]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()


Now you're ready to read in the data and use the plotting functions to visualize the data.

### Let's check 1st file: /kaggle/input/data.csv

In [0]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# data.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df1 = pd.read_csv('/kaggle/input/data.csv', delimiter=',', nrows = nRowsRead)
df1.dataframeName = 'data.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')

Let's take a quick look at what the data looks like:

In [0]:
df1.head(5)

Distribution graphs (histogram/bar graph) of sampled columns:

In [0]:
plotPerColumnDistribution(df1, 10, 5)

Correlation matrix:

In [0]:
plotCorrelationMatrix(df1, 249)

Scatter and density plots:

In [0]:
plotScatterMatrix(df1, 20, 10)

### Let's check 2nd file: /kaggle/input/labels.csv

In [0]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# labels.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df2 = pd.read_csv('/kaggle/input/labels.csv', delimiter=',', nrows = nRowsRead)
df2.dataframeName = 'labels.csv'
nRow, nCol = df2.shape
print(f'There are {nRow} rows and {nCol} columns')

Let's take a quick look at what the data looks like:

In [0]:
df2.head(5)

Distribution graphs (histogram/bar graph) of sampled columns:

In [0]:
plotPerColumnDistribution(df2, 10, 5)

## Conclusion
This concludes your starter analysis! To go forward from here, click the blue "Fork Notebook" button at the top of this kernel. This will create a copy of the code and environment for you to edit. Delete, modify, and add code as you please. Happy Kaggling!